## APLL CAPSTONE PROJECT

Note: There are two modules in this file and both work independently, module 1 can be skipped if only the deep learning module is to be used since we do not need to score the data for ranking the tickets

In [ ]:
# importing all the libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#NLTK-------------------------------
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
import math
from textblob import TextBlob
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import Sequential, model_from_json

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marlanithin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/marlanithin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/marlanithin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Importing dataset and corpus

In [ ]:
# import the dataset to be scored
data = pd.read_csv('')

### Module-1 scoring the resolution notes using nlp libraries

In [ ]:
# importing the corpus with all the word count data obtained from the original dataset
corpus = pd.read_csv('count_data_idf_lemma.csv')

In [ ]:
# calculating the idf score of each word
corpus['idf'] = corpus['count'].apply(lambda x:math.log(95698/x))

In [ ]:
# creating a dictionary to look up for the words and scores in corpus
dc_dict = corpus.set_index('word').T.to_dict('list')

/var/folders/2k/tg6w50fn00d15nxwt24cvmmc0000gn/T/ipykernel_48092/3677486083.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dc_dict = corpus.set_index('word').T.to_dict('list')


In [ ]:
# Initiating lemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk
nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/marlanithin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Function for scoring each resolution note

In [ ]:
# defining the main funcition that will score the sentences
def get_score(sentence):
    sentence = word_tokenize(sentence)
    # print(sentence)
    temp = []
    for i in sentence:
        i = str(TextBlob(i).correct())
        temp.append(lemmatizer.lemmatize(i, get_wordnet_pos(i)))
        temp.append(i)
    # print(temp)
    if len(temp)==0:
        return 0
    if len(temp)<=3:
        return 2
    score = 0
    for i in temp:
        if i in dc_dict.keys():
            score+=dc_dict[i][1]
            # print(score)
        else:
            score+=1
    score = (score/len(temp))+0.3*len(temp)
    return score
get_score('User Request')

4.555499669067901

In [ ]:
# scoring all resolution notes in the dataset
data['idf_score']=data['NotesTokenizedStemmed'].apply(lambda x: get_score2(str(x)))

### Module-2 Deep learning module application

Loading the keras model

In [ ]:
# load pickle
vectorizer = pickle.load(open("vector.pickel", "rb"))

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

In [ ]:
df = data[['Resolution notes']]

In [ ]:
review_corpus = [] # store all notes here so that we can fit the vectorizer with this dataset
y = []
for index, row in df.iterrows():
  review_text = row['Resolution notes'] 
  review_corpus.append(str(review_text))

vectorized_review = vectorizer.transform(review_corpus)

X = vectorized_review.toarray()
# predicting the satisfaction of resolution note using the loaded model
pred = loaded_model.predict(X)

In [ ]:
# combining the predictions to the data table
pred = pd.DataFrame(pred, columns=['pred'])
result = pd.concat([data, pred], axis=1)

In [ ]:
result['Root cause 1'].fillna('', inplace=True)
result['Root cause 2'].fillna('', inplace=True)
result['pred'] = result['pred'].apply(lambda x: 1 if x>0.5 else 0)

In [ ]:
# ranking the data based on the prediction and root causes
rank = []
for i, j ,l in zip(result['pred'],result['Root cause 1'],result['Root cause 2']):
  sum = 0
  if i==1:
    sum+=3
  if j:
    sum+=2
  if l:
    sum+=1
  if sum==6:
    rank.append(1)
  elif sum==4 or sum==5:
    rank.append(2)
  elif sum==3:
    rank.append(3)
  elif sum==1 or sum==2:
    rank.append(4)
  else:
    rank.append(5)

In [ ]:
rank = pd.DataFrame(rank, columns=['rank'])
result = pd.concat([result, rank], axis=1)

In [ ]:
# saving the file
result.to_csv('ranked_data.csv',index=False)